### eSIM profiles
- Подготовка файла с eSIM профайлами для выдачи Касмтомерам

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import datetime as dt
from os.path import join

import roamability as rb

In [2]:
downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA\eSimProfiles\data'

# keys_files = ['esim_rb_5k_18102019.out', 'RB_200804_5K_eSIM.out']
lpa_files = ['esim_rb_5k_18102019.csv', 'RB_200804_5K_eSIM.csv', 'Freecell_200807_5K_eSIM.csv']

df_qr = DataFrame(columns = ['ICCID', 'QR_Code'])

for file in lpa_files:
    df_qr = df_qr.append(pd.read_csv(join(downloads, file), engine='python', dtype='str'))
    
# keys_cols = ['IMSI','PIN1','PUK1','PIN2','PUK2','KIC1','KID1','KIK1']
# df_keys = DataFrame(columns = keys_cols)

# for file in keys_files:
#     df_keys = df_keys.append(pd.read_csv(join(downloads, file), sep=' ', engine='python', dtype='str',
#                                         usecols=keys_cols))

df_qr['ICCID_CD'] = df_qr.ICCID.str.slice(0,19)
df_qr.sort_values(by='ICCID_CD', inplace=True)
df_qr.reset_index(inplace=True, drop=True)

display(df_qr.head(3))

,ICCID,QR_Code,ICCID_CD
0,8997212330099119820,LPA:1$smdp.io$4K-T4R38-P9G6S6,8997212330099119820
1,8997212330099119821,LPA:1$smdp.io$4K-T4R3F-DCQ6YC,8997212330099119821
2,8997212330099119822,LPA:1$smdp.io$4K-T4R3M-101NJ6N,8997212330099119822


In [3]:
# Из базы данных OCS выгружаем IMSI
#----------------------------------

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT
r.RESELLER_NAME
,a.ACCOUNT_NAME
,ss.STATUS AS SUB_STATUS
,si.IMSI
,si.ICCID AS ICCID_CD
FROM RESELLERS r
INNER JOIN ACCOUNTS a
ON r.RESELLER_ID = a.RESELLER_ID
INNER JOIN SUBSCRIBERS sr
ON sr.ACCOUNT_ID = a.ACCOUNT_ID
INNER JOIN SUBSCRIBER_STATUS ss
ON ss.SUBSCRIBER_ID = sr.SUBSCRIBER_ID
INNER JOIN SUBSCRIBER_IMSIS si
ON ss.SUBSCRIBER_ID = si.SUBSCRIBER_ID
WHERE
r.deleted is NULL
AND ss.END_DATE IS NULL
--AND ss.status IN ('Active', 'Disconnected', 'Manually Suspended')
AND
(
si.IMSI BETWEEN '260060145010000' AND '260060145014999' OR
si.IMSI BETWEEN '260060145031000' AND '260060145035999' OR
si.IMSI BETWEEN '260060145026010' AND '260060145030999'
)
'''

df_ocs_imsi = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_ocs_imsi['IMSI'] = df_ocs_imsi['IMSI'].astype('int64')
cnxn.close()
df_ocs_imsi.head(3)

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,ICCID_CD
0,Drimsim,Drimsim eSIM,Active,260060145014246,8997219121000091246
1,Drimsim,Drimsim eSIM,Active,260060145014247,8997219121000091247
2,Drimsim,Drimsim eSIM,Active,260060145014248,8997219121000091248


In [4]:
cols = ['ICCID','QR_Code','RESELLER_NAME','ACCOUNT_NAME', 'IMSI', 'SUB_STATUS']
df_batch = pd.merge(df_qr, df_ocs_imsi, on='ICCID_CD', how='left', suffixes=['_S2', '_S1'])

df_batch = df_batch[cols]
df_batch.head()

,ICCID,QR_Code,RESELLER_NAME,ACCOUNT_NAME,IMSI,SUB_STATUS
0,8997212330099119820,LPA:1$smdp.io$4K-T4R38-P9G6S6,Freecell,Freecell_200807_5K_eSIM,260060145026010,Active
1,8997212330099119821,LPA:1$smdp.io$4K-T4R3F-DCQ6YC,Freecell,Freecell_200807_5K_eSIM,260060145026011,Active
2,8997212330099119822,LPA:1$smdp.io$4K-T4R3M-101NJ6N,Freecell,Freecell_200807_5K_eSIM,260060145026012,Active
3,8997212330099119823,LPA:1$smdp.io$4K-T4R3T-1X2K5XY,Freecell,Freecell_200807_5K_eSIM,260060145026013,Active
4,8997212330099119824,LPA:1$smdp.io$4K-T4R47-1HTFLL9,Freecell,Freecell_200807_5K_eSIM,260060145026014,Active


In [5]:
list_agg = [('IMSI_MIN','min'),('IMSI_MAX','max'),('NUM','count')]
list_col = ['RESELLER_NAME', 'ACCOUNT_NAME', 'SUB_STATUS', df_batch.IMSI.astype('str').str.slice(0,12)]
df_batch_group = df_batch.groupby(list_col)['IMSI'].agg(list_agg).reset_index().sort_values(by='IMSI')
df_batch_group.sort_values('IMSI_MIN')

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
51,WMB Limited RT,WMB eSIMs,Active,260060145010,260060145010000,260060145010999,1000
33,STI - Telinta,ESIM_SW_Main_Account,Active,260060145011,260060145011000,260060145011839,839
32,STI - Telinta,ESIM_KALLER,Active,260060145011,260060145011834,260060145011834,1
31,STI - Telinta,ESIM_GlobaleSIM_Website,Active,260060145011,260060145011840,260060145011899,60
29,STI - Telinta,ESIM_Adonis,Active,260060145011,260060145011900,260060145011949,50
30,STI - Telinta,ESIM_GVC,Active,260060145011,260060145011950,260060145011999,50
43,Simtex,Simtex eSIMs - IO,Active,260060145012,260060145012000,260060145012998,461
42,Simtex,Mobimatter,Active,260060145012,260060145012121,260060145012796,529
45,Simtex,eSIM test,Active,260060145012,260060145012557,260060145012557,1
44,Simtex,UAE,Active,260060145012,260060145012848,260060145012849,2


In [6]:
df_batch_group.loc[df_batch_group.RESELLER_NAME == 'Roamability Invertory'].sort_values('IMSI_MIN')
# df_batch_group.loc[df_batch_group.RESELLER_NAME == 'STI - Telinta'].sort_values('IMSI_MIN')\
# .to_csv(join(downloads, 'Freecell.csv'), index=False)

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
20,Roamability Invertory,Roamability eSIM Inventory,Active,260060145014,260060145014989,260060145014999,11
18,Roamability Invertory,Freecell_200807_5K_eSIM,Active,260060145030,260060145030900,260060145030999,100
19,Roamability Invertory,RB_200804_5K_eSIM,Active,260060145035,260060145035267,260060145035998,732


In [7]:
# Prepare the output file

qustomer = 'ImsiMarket'
imsi_start = 260060145035267
given_quantity = 3

current_date = dt.datetime.now().strftime('%y%m%d')

of_name = f'RB_eSIM_{qustomer}_{current_date}_{given_quantity}.csv'
df_customer = df_batch.loc[df_batch.IMSI.isin(range(imsi_start, imsi_start + given_quantity)),
       ['ICCID','QR_Code','IMSI']]

df_customer.QR_Code.replace(to_replace={r'\.*rsp.truphone.com\.*':'smdp.io'}, regex=True, inplace=True)

df_customer.to_csv(join(downloads, of_name), index=False)
df_customer.head()

,ICCID,QR_Code,IMSI
9257,8997212330099156077,LPA:1$smdp.io$4K-U8RVH-110JQHD,260060145035267
9258,8997212330099156078,LPA:1$smdp.io$4K-U8RVO-QSLB3J,260060145035268
9259,8997212330099156079,LPA:1$smdp.io$4K-U8RVV-F00B81,260060145035269


### Prepare the output file for Customer order

In [8]:
#######################################################################################
# Define variables here
#######################################################################################

esim_out_file_workz = 'STI_201210_5_5_eSIM_S1_matchingid.out'

# Format like:
# 89972123300991198102 4K-T4G53-SAL8OK
# 89972123300991198110 4K-T4G5A-GRZD8A
# 89972123300991198128 4K-T4G5H-299K5X

iccid_beg = 8997212330099192000
iccid_end = 8997212330099194999

order_name = 'STI_201210_5_5_eSIM_S1_Single' # to use as the name of output file for Customer

#######################################################################################

#######################################################################################

In [9]:
# Download the data from the output file from Workz

df_esim = pd.read_csv(join(downloads, esim_out_file_workz), sep=' ', names=['ICCID_C', 'ESIM'])
df_esim['ICCID'] = df_esim.ICCID_C.str.slice(0, 19)
lpa = 'LPA:1$smdp.io$'
df_esim['QR_Code'] = lpa + df_esim.ESIM
df_esim.drop(['ICCID_C', 'ESIM'], axis=1, inplace=True)

df_esim.head()

,ICCID,QR_Code
0,8997212330099192000,LPA:1$smdp.io$4K-13W6V0-18NI9JL
1,8997212330099192001,LPA:1$smdp.io$4K-13W6VE-RPW1PY
2,8997212330099192002,LPA:1$smdp.io$4K-13W6VZ-1CEVK6M
3,8997212330099192003,LPA:1$smdp.io$4K-13W6WD-C2BNL3
4,8997212330099192004,LPA:1$smdp.io$4K-13W6WR-174CDM


In [10]:
# Collect subscriber's data from OCS

sql_srt = """
SELECT
a.account_name AS ACCOUNT
,si.ICCID
,spn.phone_number AS MSISDN
,si.IMSI
FROM SUBSCRIBER_IMSIS si
LEFT JOIN SUBSCRIBER_STATUS ss ON si.subscriber_id = ss.subscriber_id
LEFT JOIN SUBSCRIBERS s ON si.subscriber_id = s.subscriber_id
LEFT JOIN ACCOUNTS a ON a.account_id = s.account_id
LEFT JOIN SUBSCRIBER_PHONE_NUMBERS spn ON si.subscriber_id = spn.subscriber_id
WHERE ss.STATUS IN ('Active', 'Suspended', 'Pending') AND ss.END_DATE IS NULL
AND si.iccid BETWEEN '{}' AND  '{}'
ORDER BY si.ICCID""".format(iccid_beg, iccid_end)

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_ocs = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

display(df_ocs.groupby('ACCOUNT')['ICCID','IMSI'].agg([('MIN',min),('MAX', max),('NUM','count')]))

df_ocs.head()

ICCID                       \
                                                     MIN                  MAX   
ACCOUNT                                                                         
STI_201210_5_5_eSIM_S1_S4_S6_Single  8997212330099192000  8997212330099194999   

                                                      IMSI                   \
                                      NUM              MIN              MAX   
ACCOUNT                                                                       
STI_201210_5_5_eSIM_S1_S4_S6_Single  3000  425019614027000  425019614029999   

                                           
                                      NUM  
ACCOUNT                                    
STI_201210_5_5_eSIM_S1_S4_S6_Single  3000

,ACCOUNT,ICCID,MSISDN,IMSI
0,STI_201210_5_5_eSIM_S1_S4_S6_Single,8997212330099192000,972541320000,425019614027000
1,STI_201210_5_5_eSIM_S1_S4_S6_Single,8997212330099192001,972541320001,425019614027001
2,STI_201210_5_5_eSIM_S1_S4_S6_Single,8997212330099192002,972541320002,425019614027002
3,STI_201210_5_5_eSIM_S1_S4_S6_Single,8997212330099192003,972541320003,425019614027003
4,STI_201210_5_5_eSIM_S1_S4_S6_Single,8997212330099192004,972541320004,425019614027004


In [11]:
# Megre DataFrames and save to file

pd.merge(df_ocs, df_esim, how='outer', on='ICCID')\
.to_csv(join(downloads, f'{order_name}.csv'), index=False)

### Developments

In [4]:
downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA\eSimProfiles\data'

# keys_files = ['esim_rb_5k_18102019.out', 'RB_200804_5K_eSIM.out']
lpa_files = ['esim_rb_5k_18102019.csv', 'RB_200804_5K_eSIM.csv', 'Freecell_200807_5K_eSIM.csv']

df_qr = DataFrame(columns = ['ICCID', 'QR_Code', 'FName'])

for file in lpa_files:
    df_temp = pd.read_csv(join(downloads, file), engine='python', dtype='str')
    df_temp['FName'] = file
    df_qr = df_qr.append(df_temp)
    
# keys_cols = ['IMSI','PIN1','PUK1','PIN2','PUK2','KIC1','KID1','KIK1']
# df_keys = DataFrame(columns = keys_cols)

# for file in keys_files:
#     df_keys = df_keys.append(pd.read_csv(join(downloads, file), sep=' ', engine='python', dtype='str',
#                                         usecols=keys_cols))

df_qr['ICCID_CD'] = df_qr.ICCID.str.slice(0,19)
df_qr.sort_values(by='ICCID_CD', inplace=True)
df_qr.reset_index(inplace=True, drop=True)

display(df_qr.head(3))

,ICCID,QR_Code,FName,ICCID_CD
0,8997212330099119820,LPA:1$smdp.io$4K-T4R38-P9G6S6,Freecell_200807_5K_eSIM.csv,8997212330099119820
1,8997212330099119821,LPA:1$smdp.io$4K-T4R3F-DCQ6YC,Freecell_200807_5K_eSIM.csv,8997212330099119821
2,8997212330099119822,LPA:1$smdp.io$4K-T4R3M-101NJ6N,Freecell_200807_5K_eSIM.csv,8997212330099119822


In [5]:
cols = ['ICCID','QR_Code','RESELLER_NAME','ACCOUNT_NAME', 'IMSI', 'SUB_STATUS', 'FName']
df_batch = pd.merge(df_qr, df_ocs_imsi, on='ICCID_CD', how='left', suffixes=['_S2', '_S1'])

df_batch = df_batch[cols]
df_batch.head()

,ICCID,QR_Code,RESELLER_NAME,ACCOUNT_NAME,IMSI,SUB_STATUS,FName
0,8997212330099119820,LPA:1$smdp.io$4K-T4R38-P9G6S6,Freecell,Freecell_200807_5K_eSIM,260060145026010,Active,Freecell_200807_5K_eSIM.csv
1,8997212330099119821,LPA:1$smdp.io$4K-T4R3F-DCQ6YC,Freecell,Freecell_200807_5K_eSIM,260060145026011,Active,Freecell_200807_5K_eSIM.csv
2,8997212330099119822,LPA:1$smdp.io$4K-T4R3M-101NJ6N,Freecell,Freecell_200807_5K_eSIM,260060145026012,Active,Freecell_200807_5K_eSIM.csv
3,8997212330099119823,LPA:1$smdp.io$4K-T4R3T-1X2K5XY,Freecell,Freecell_200807_5K_eSIM,260060145026013,Active,Freecell_200807_5K_eSIM.csv
4,8997212330099119824,LPA:1$smdp.io$4K-T4R47-1HTFLL9,Freecell,Freecell_200807_5K_eSIM,260060145026014,Active,Freecell_200807_5K_eSIM.csv


In [6]:
df_batch.loc[df_batch.IMSI==260060145033219]

,ICCID,QR_Code,RESELLER_NAME,ACCOUNT_NAME,IMSI,SUB_STATUS,FName
7209,8997212330099154029,LPA:1$smdp.io$4K-U8EDK-VBEOWC,STI - Telinta,New SIM cards,260060145033219,Active,RB_200804_5K_eSIM.csv
